In [32]:
import torch
import torch.nn as nn
import torch.nn.init
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [33]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## 합성곱 신경망(Convolutional Neural Network)

`합성곱 신경망(Convolutional Neural Network)`은 주로 `이미지 처리`에 사용되고 뛰어난 성능을 보이는 신경망

합성곱 신경망은 `합성곱층(Convolution layer)`, `풀링층(Pooling layer)`으로 구성, ReLU 같은 활성화 함수 또한 같이 사용됨

퍼셉트론은 이미지 입력을 1차원 텐서로 변환 해야하고 그에 따라 변환 전에 가지고 있던 공간적인 구조(spatial structure) 정보가 유실됨

이미지의 공간적인 구조 정보를 보존하면서 학습할 수 있는 합성곱 신경망이 이미지 처리에 주로 사용됨

### 채널(Channel)

이미지는 (높이, 너비, 채널) 형태의 3차원 텐서

채널은 이미지의 색 성분, 따라서 흑백(gray) 이미지는 1채널, RGB 채널을 사용하는 컬러 이미지는 3채널을 가짐

채널은 `깊이(depth)` 라고도 함

### 커널(kernel)

`필터(filter)` 라고도 함

커널은 n * m 크기의 행렬로 이후 합성곱 연산에 사용됨

커널은 일반적으로 3 * 3 혹은 5 * 5 가 주로 사용됨

커널의 형태, 즉 커널 내부의 구성 값에 따라 합성곱 연산 결과에 여러가지 변화를 줄 수 있음

### 합성곱 연산(Convolution operation)

`합성곱층(Convolution layer)` 에서 수행되는 연산

커널을 통해 이미지에서 특징을 추출하는 역할을 함

커널로 이미지를 훑으면서 커널과 겹쳐지는 부분을 커널의 원소값과 곱한 후 모두 더한 값을 출력으로 사용

즉 커널의 값이 퍼셉트론에서 사용한 가중치와 같다고 볼 수 있음

커널을 사용하여 합성곱 연산을 통해 나온 결과를 `특성 맵(feature map)`이라 함

각 연산 후 커널의 이동 범위를 `스트라이드(stride)`라 함

### 패딩(Padding)

5 * 5 이미지에 3 * 3 커널로 스트라이드가 1인 컨볼루션 연산을 하면 3 * 3의 피처 맵을 얻을 수 있다.

이 처럼 피처 맵은 커널의 크기의 비례하여 입력 이미지보다 크기가 작아진다.

이 경우 여러 합성곱 층을 지나면 이미지가 입력보다 작아지게 되므로 피처 맵의 크기를 유지할 필요가 있다

`패딩(padding)`은 입력 이미지의 가장자리에 특정 개수의 폭만큼 값을 추가해주는 것이다

5 * 5 입력 이미지에 `제로 패딩(zero padding)`을 사용하면 가장자리에 0이 추가되어, 7 * 7 의 이미지로 변환 된다.

### 가중치와 편향

특성 맵을 얻기 위해서는 동일한 커널로 이미지 전체를 훑으며 합성곱 연산을 진행

결국 해당 연산에서 사용되는 가중치는 커널의 원소 값들로 3 * 3 커널이라면 9개이다

합성곱 신경망은 MLP보다 훨씬 적은 수의 가중치를 사용하며, 공간적 구조 정보를 보존한다

편향(bias)을 사용한다면 커널을 적용한뒤 편향을 더해서 피처맵을 만든다

편향은 하나의 값만 존재하고 커널이 적용된 결과의 모든 원소에 더한다

CNN에서도 비선형성을 추가하기 위해서 활성화 함수를 통과함

주로 ReLU 와 그 변형을 사용함

합성곱 연산을 통해서 특성 맵을 얻고, 활성화 함수를 지나는 연산을 하게되는데

해당 은닉층을 합성곱 층(convolution layer)이라 한다

### 풀링(Pooling)

일반적으로 합성곱 층(합성곱 연산 + 활성화 함수) 다음에는 풀링 층을 추가하는 것이 일반적

풀링 층에서는 특성 맵을 다운샘플링하여 특성 맵의 크기를 줄이는 풀링 연산을 수행

풀링 연산에서도 합성곱 연산과 마찬가지로 커널과 스트라이드의 개념을 가짐

풀링 연산에는 일반적으로 `최대 풀링(max pooling)`과 `평균 풀링(average pooling)`이 사용

각각 커널 영역과 겹치는 원소들의 최대값/평균값을 가져오는 연산

합성곱 연산과의 차이점은 학습해야 할 가중치가 없으며 연산 후에 채널 수가 변하지 않는다는 점

### 특성 맵의 크기

입력의 크기와 커널의 크기, 그리고 스트라이드의 값만 알면 합성곱 연산의 결과인 특성 맵의 크기를 계산 가능

입력 I의 높이 h, 너비 w

커널 K의 높이 h, 너비 w

특성 맵 O의 높이 h, 너비 w

스트라이드 S, 패딩의 폭 P 라고 할때 특성맵의 크기는

$ O_h = floor(\frac{I_h - K_h + 2P}{S} + 1) $

$ O_w = floor(\frac{I_w - K_w + 2P}{S} + 1) $

### 3차원 텐서의 합성곱 연산

채널(channel) 또는 깊이(depth)를 고려한 3차원 텐서의 경우의 합성곱 연산은 채널 수 만큼 연산을 추가로 수행함

즉 다수의 채널을 가진 입력 데이터를 가지고 합성곱 연산을 한다고 하면, 커널의 채널 수도 입력의 채널 수만큼 존재

입력 데이터의 채널 수와 커널의 채널 수는 같아야 함

합성곱 연산을 채널마다 수행 후 그 결과를 모두 더하여 최종 특성 맵을 구함

입력 데이터의 채널을 $C_i$ 라 하면, 3차원 텐서의 합성곱 연산은

입력 데이터 * 커널 = 특성 맵

$ (I_h \times I_w \times C_i) * (K_h \times K_w \times C_i) = (O_h \times O_w) $

채널 1의 특성맵을 구할 수 있음, 여러개의 커널을 사용하는 경우 커널 수를 $C_o$라 하면

$ (I_h \times I_w \times C_i) * ((K_h \times K_w \times C_i) \times C_o) = (O_h \times O_w \times C_o) $

합성곱 연산에서 다수의 커널을 사용할 경우, 사용한 커널 수는 합성곱 연산의 결과로 나오는 특성 맵의 채널 수가 됨

가중치 매개변수는 커널에만 존재 하므로 다수 커널을 사용했을때 커널의 크기가 가중치 매개변수의 총 수와 같다

따라서 가중치 매개변수 총 수 : $ K_h \times K_w \times C_i \times C_o $

## CNN으로 MNIST 분류

In [34]:
# 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [35]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

In [36]:
print(mnist_train)
print(mnist_test)

Dataset MNIST
    Number of datapoints: 60000
    Root location: MNIST_data/
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: MNIST_data/
    Split: Test
    StandardTransform
Transform: ToTensor()


In [37]:
print(mnist_train.data[0].shape)
print(mnist_train.targets[0].shape)

torch.Size([28, 28])
torch.Size([])


In [38]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [39]:
total_batch = len(data_loader)
print(total_batch)

600


In [40]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out
    
model = CNN().to(device)

In [41]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [42]:
def train(epochs):
    for epoch in range(epochs):
        avg_cost = 0

        for X, Y in data_loader: # X는 미니 배치, Y는 레이블
            # image is already size of (28x28), no reshape
            # label is not one-hot encoded
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()
            hypothesis = model(X)
            cost = criterion(hypothesis, Y)
            cost.backward()
            optimizer.step()

            avg_cost += cost / total_batch

        print(f'[Epoch: {epoch + 1:>4}] cost = {avg_cost:>.9}')

In [43]:
def test():
    with torch.no_grad():
        X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
        Y_test = mnist_test.test_labels.to(device)

        prediction = model(X_test)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

In [44]:
test()

Accuracy: 0.12709999084472656


In [45]:
train(training_epochs)

[Epoch:    1] cost = 0.234325916
[Epoch:    2] cost = 0.0639880598
[Epoch:    3] cost = 0.048919864
[Epoch:    4] cost = 0.0390038788
[Epoch:    5] cost = 0.0317291617
[Epoch:    6] cost = 0.0274743121
[Epoch:    7] cost = 0.0241682176
[Epoch:    8] cost = 0.01941384
[Epoch:    9] cost = 0.0178485885
[Epoch:   10] cost = 0.0147665823
[Epoch:   11] cost = 0.0127897914
[Epoch:   12] cost = 0.0103323692
[Epoch:   13] cost = 0.0095290076
[Epoch:   14] cost = 0.00779245235
[Epoch:   15] cost = 0.00677087065


In [46]:
test()

Accuracy: 0.986299991607666
